<a href="https://colab.research.google.com/github/gvxfnv/lizavavilova/blob/main/%D0%B4%D0%BE_25_%D0%B0%D0%BF%D1%80%D0%B5%D0%BB%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Вариант 7. Учет и управление фитнес-клубом

In [1]:
from abc import ABC, abstractmethod
from datetime import date

class Member(ABC):
    def __init__(self, member_id: int, name: str, age: int, membership_type: str, join_date: date):
        self.__member_id = member_id
        self.__name = name
        self.__age = age
        self.__membership_type = membership_type
        self.__join_date = join_date

    @abstractmethod
    def get_membership_info(self) -> str:
        pass

    def __str__(self):
        return f"Участник: {self.__name}, Тип членства: {self.__membership_type}"

    def __lt__(self, other):
        return self.__age < other.__age

    def __gt__(self, other):
        return self.__age > other.__age

    # Геттеры
    @property
    def member_id(self) -> int:
        return self.__member_id

    @property
    def name(self) -> str:
        return self.__name

    @property
    def age(self) -> int:
        return self.__age

    @property
    def membership_type(self) -> str:
        return self.__membership_type

    @property
    def join_date(self) -> date:
        return self.__join_date
class Client(Member):
    def __init__(self, member_id: int, name: str, age: int, membership_type: str, join_date: date, subscription: str):
        super().__init__(member_id, name, age, membership_type, join_date)
        self.__subscription = subscription

    def get_membership_info(self) -> str:
        return f"Клиент: {self.name}, Абонемент: {self.__subscription}"

    def __str__(self):
        return f"{super().__str__()}, Абонемент: {self.__subscription}"

class Trainer(Member):
    def __init__(self, member_id: int, name: str, age: int, membership_type: str, join_date: date, specialization: str):
        super().__init__(member_id, name, age, membership_type, join_date)
        self.__specialization = specialization

    def get_membership_info(self) -> str:
        return f"Тренер: {self.name}, Специализация: {self.__specialization}"

    def __str__(self):
        return f"{super().__str__()}, Специализация: {self.__specialization}"
class Location:
    def __init__(self, address: str):
        self.address = address

class GymClass:
    def __init__(self, class_name: str, trainer: Trainer, schedule: str, location: Location):
        self.class_name = class_name
        self.trainer = trainer
        self.schedule = schedule
        self.location = location
        self.participants = []

    def add_participant(self, participant: Member):
        self.participants.append(participant)

    def remove_participant(self, participant: Member):
        self.participants.remove(participant)

    def get_participants(self):
        return [str(p) for p in self.participants]
from abc import ABC

class Bookable(ABC):
    @abstractmethod
    def book_class(self, participant: Member) -> None:
        pass

class Reportable(ABC):
    @abstractmethod
    def generate_report(self) -> str:
        pass
class LoggingMixin:
    def log_action(self, action: str) -> None:
        print(f"Логирование действия: {action}")

class NotificationMixin:
    def send_notification(self, message: str) -> None:
        print(f"Уведомление: {message}")
class MemberMeta(type):
    members_registry = {}

    def __new__(cls, name, bases, attrs):
        new_class = super().__new__(cls, name, bases, attrs)
        cls.members_registry[name] = new_class
        return new_class
class MemberFactory:
    @staticmethod
    def create_member(member_type: str, *args):
        if member_type == "client":
            return Client(*args)
        elif member_type == "trainer":
            return Trainer(*args)
        else:
            raise ValueError("Неизвестный тип участника")
class Handler(ABC):
    def set_next(self, handler):
        self.next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request):
        if hasattr(self, 'next_handler'):
            return self.next_handler.handle(request)
        return None

class Administrator(Handler):
    def handle(self, request):
        if request['duration'] == 1:
            print("Администратор одобрил продление на 1 месяц.")
            return True
        else:
            return super().handle(request)

class Manager(Handler):
    def handle(self, request):
        if request['duration'] == 3:
            print("Менеджер одобрил продление на 3 месяца.")
            return True
        else:
            return super().handle(request)
class Director(Handler):
    def handle(self, request):
        print("Директор одобрил продление на {} месяцев.".format(request['duration']))
        return True
class SubscriptionRenewal:
    def renew_subscription(self, request):
        if not self.validate_request(request):
            print("Запрос на продление не валиден.")
            return False
        return self.process_renewal(request)
    def validate_request(self, request):
        return True
    def process_renewal(self, request):
        raise NotImplementedError("Метод должен быть переопределен в подклассе.")
class ConcreteRenewal(SubscriptionRenewal):
    def process_renewal(self, request):
        print("Обработка продления абонемента...")
if __name__ == "__main__":
    client = MemberFactory.create_member("client", 1, "Иван", 25, "Стандарт", date.today(), "Месячный")
    trainer = MemberFactory.create_member("trainer", 2, "Алексей", 30, "Профессионал", date.today(), "Фитнес")
    print(client.get_membership_info())
    print(trainer.get_membership_info())
    location = Location("Зал 1")
    gym_class = GymClass("Силовая тренировка", trainer, "Пн 10:00", location)
    gym_class.add_participant(client)
    print(gym_class.get_participants())
    admin = Administrator()
    manager = Manager()
    director = Director()
    admin.set_next(manager).set_next(director)
    renewal_request = {"duration": 1}
    admin.handle(renewal_request)

Клиент: Иван, Абонемент: Месячный
Тренер: Алексей, Специализация: Фитнес
['Участник: Иван, Тип членства: Стандарт, Абонемент: Месячный']
Администратор одобрил продление на 1 месяц.
